In [ ]:
import numpy as np
npz = np.load('/home/e19b516g/yejing/data/data_for_graph/npz/val/form_5_648_E3236.npz')

In [ ]:
print(npz['edge_labels'])

In [ ]:
import torch
a = torch.tensor([[0,1],[1,0]])
# fill the diagonal with 1
a.fill_diagonal_(1)

In [ ]:
print(npz['strokes_emb'].shape)

In [ ]:
print(npz['stroke_labels'].shape)

In [ ]:
import torch
nn = torch.empty(0, 100, 2)

In [ ]:
emb = torch.ones(10,100,2)
nn = torch.cat((nn, emb), 0)

In [ ]:
nn = torch.cat((nn, torch.zeros(3,100,2)), 0)

In [ ]:
nn.shape

In [ ]:
y = torch.load('/home/e19b516g/yejing/data/data_for_graph/S100_R5_Speed_False/val_y.pt')
X = torch.load('/home/e19b516g/yejing/data/data_for_graph/S100_R5_Speed_False/val_X.pt')

In [ ]:
print(y.shape)
print(X.shape)

In [ ]:
from tsai.all import *
s = get_splits(y, valid_size=0.2, random_state=42)

In [ ]:
s

In [ ]:
# create tuple sss[1] 0~100, sss[2] 101~200
sss = (list(range(0,12)), list(range(12,20)))

In [ ]:
sss

In [ ]:
import torch

ckpt = torch.load('/home/e19b516g/yejing/code/Edge_GAT/pretrain_logs/S100_R5_Speed_False_lr_0.001/version_1/checkpoints/epoch=22-step=120037.ckpt',map_location=torch.device('cpu'))

In [ ]:
from Model.LitModel import LitModel
model = LitModel(
    node_input_size = 100,
    edge_input_size = 20,
    gat_input_size = 128,
    gat_hidden_size = 64,
    gat_output_size = 128,
    gat_n_heads = 8,
    node_class_nb = 114,
    edge_class_nb = 14,
    dropout = 0.6,
    lambda1 = 0.5,
    lambda2 =  0.5,
    lr = 1e-3,
    device = 'cpu'
)
# model.load_state_dict(ckpt, strict=False)

In [ ]:
model.load_state_dict(ckpt, strict=False)

In [ ]:

for name, param in model.named_parameters():
    print(name, param.shape)

In [ ]:
# new_state_dict = OrderedDict()
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in ckpt['state_dict'].items():
    # print(k, v.shape)
    # print(k[:6] + 'node_emb.' + k[6:])
    new_state_dict[k[:6] + 'node_emb.' + k[6:]] = v

In [ ]:
import pre_train
model_x = pre_train.LightModel()
model_x.load_state_dict(new_state_dict, strict=False)

In [ ]:
test = torch.tensor([[1,2,3],[4,5,6]])
test.unsqueeze(-1).shape

In [ ]:
import numpy as np
import os

In [ ]:
npz_path = '/home/e19b516g/yejing/data/data_for_graph/npz/train'
node_nb = []
edge_nb = []
for _, _, files in os.walk(npz_path):
    for file in files:
        if file.endswith('.npz'):
            npz =np.load(os.path.join(npz_path, file))
            node_nb.append(npz['strokes_emb'].shape[0])
            edge_nb.append(np.sum(npz['los'] == 1))

In [ ]:
nnn = np.load('/home/e19b516g/yejing/data/data_for_graph/npz/train/001-equation000.npz')
nnn['edge_labels'].shape

In [ ]:
import matplotlib.pyplot as plt
plt.hist(np.array(node_nb), bins=70)

In [ ]:
node_nb = np.array(node_nb)
node_nb.max()

In [ ]:
plt.hist(np.array(edge_nb), bins=50)

In [ ]:
def balance_graphs(node_counts, edge_counts, nodes_per_group):
    # Calculate the total number of nodes and groups
    total_nodes = sum(node_counts)
    num_groups = total_nodes // nodes_per_group
    
    # Initialize groups
    groups = [[] for _ in range(num_groups)]
    
    # Create a list of (index, node_count, edge_count) tuples
    graphs = list(enumerate(zip(node_counts, edge_counts)))
    
    # Sort graphs in descending order of edge_counts
    sorted_graphs = sorted(graphs, key=lambda x: x[1][1], reverse=True)
    
    # Distribute graphs into groups
    for index, (nodes, edges) in sorted_graphs:
        for group in groups:
            if sum(graphs[i][1][0] for i in group) + nodes <= nodes_per_group:
                group.append(index)
                break
    
    return groups

# Example usage:
# node_counts = [10, 15, 8, 12, 7]  # List of node counts
# edge_counts = [20, 30, 25, 18, 14]  # List of edge counts
# num_groups = 2  # Set the number of groups



In [ ]:
import itertools

def balance_graphs(node_counts, edge_counts, nodes_per_group):
    num_groups = sum(node_counts) // nodes_per_group
    num_graphs = len(node_counts)

    # Calculate the total number of edges
    total_edges = sum(edge_counts)

    # Sort graphs by edge counts in ascending order
    sorted_graphs = sorted(enumerate(edge_counts), key=lambda x: x[1])

    # Initialize group assignments
    group_assignments = [-1] * num_graphs

    # Initialize group statistics
    group_node_sums = [0] * num_groups
    group_edge_sums = [0] * num_groups

    for graph_idx, edge_count in sorted_graphs:
        # Try to add the graph to the group with the smallest sum of nodes
        min_group = group_node_sums.index(min(group_node_sums))
        if group_assignments[graph_idx] == -1:
            if group_node_sums[min_group] + node_counts[graph_idx] <= nodes_per_group:
                group_assignments[graph_idx] = min_group
                group_node_sums[min_group] += node_counts[graph_idx]
                group_edge_sums[min_group] += edge_count

    # If there are unassigned graphs, assign them to the group with the smallest edge sum
    for graph_idx, edge_count in sorted_graphs:
        if group_assignments[graph_idx] == -1:
            min_group = group_edge_sums.index(min(group_edge_sums))
            group_assignments[graph_idx] = min_group
            group_edge_sums[min_group] += edge_count

    # Organize the groups
    groups = [[] for _ in range(num_groups)]
    for graph_idx, group_idx in enumerate(group_assignments):
        groups[group_idx].append(graph_idx)
    return groups



In [ ]:
groups = balance_graphs(node_nb, edge_nb, 128)

for i, group in enumerate(groups):
    node_sum = 0
    edge_sum = 0
    print(f'Group {i}:')
    for index in group:
        nodes, edges = node_nb[index], edge_nb[index]
        node_sum += nodes
        edge_sum += edges
        print(f'  Graph {index + 1} with {nodes} nodes and {edges} edges')
    print(f'  Total: {node_sum} nodes and {edge_sum} edges\n')


In [ ]:
from Dataset.Datamodule import CROHMEDatamodule
dm = CROHMEDatamodule(
    root_path = npz_path,
    shuffle = True,
    num_workers = 0,
    reload_dataloaders_every_n_epochs=1
)


In [ ]:
list = ['N2E20_form_5_673_E3363', 'N5E29_form_5_673_E3363', 'N25E290_form_5_673_E3363']
# get number behind N and E in the list
import re
node_nb = []
edge_nb = []
for i in list:
    n = i.split('_')[0]
    node_nb.append(n.split('E')[0].split('N')[1])
    edge_nb.append(n.split('E')[1])

In [ ]:
edge_nb

In [ ]:
x_rand = torch.rand(3, 3, 4, 5)
xx = torch.zeros(0, 0, 4, 5)
xx = torch.cat((xx, x_rand), dim=2)

In [ ]:
import torch
g_ = torch.rand(3, 8, 64)
g_r = g_.repeat(3, 1, 1)
g_r_i = g_.repeat_interleave(3, dim=0)
print(g_r.shape)
print(g_r_i.shape)

In [ ]:
b = torch.rand(9, 8, 32)

In [ ]:
g_con = torch.cat([g_r, g_r_i], dim=-1)

In [ ]:
torch.cat([g_r, g_r_i, b], dim=-1).shape

In [ ]:
import numpy as np
npz = np.load('/home/e19b516g/yejing/data/data_for_graph/S150_R10/train/N15E128_form_023_E178.npz')
print(npz['edge_labels'])

In [ ]:
los = npz['los']
el = npz['edge_labels']
los = torch.from_numpy(los)
el = torch.from_numpy(el)
los = torch.triu(los)
indices = torch.nonzero(los.reshape(-1))
el = el.reshape(-1)[indices]

In [ ]:
len(indices)

In [ ]:
los.shape

In [ ]:
import torch
pt = torch.load('/home/e19b516g/yejing/code/Edge_GAT/val_results/2024_01_05_06_13_30/version_0/epoch_90.pt', map_location=torch.device('cpu'))

In [ ]:
node_pred = torch.zeros(0)
edge_pred = torch.zeros(0)
node_label = torch.zeros(0)
edge_label = torch.zeros(0)
for i in pt:
    node_hat = torch.argmax(i[0], dim=1)
    node_pred = torch.cat((node_pred, node_hat), dim=0)
    stroke_label = i[1]
    node_label = torch.cat((node_label, stroke_label), dim=0)
    edge_hat = torch.argmax(i[2], dim=1)
    edge_pred = torch.cat((edge_pred, edge_hat), dim=0)
    rel_label = i[3]
    edge_label = torch.cat((edge_label, rel_label), dim=0)



In [ ]:
indices = torch.nonzero(edge_label.reshape(-1))
edge_label = edge_label.reshape(-1)[indices]
edge_pred = edge_pred.reshape(-1)[indices]

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(node_label, node_pred))
print(accuracy_score(edge_label, edge_pred))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(edge_label, edge_pred)
cm_node = confusion_matrix(node_label, node_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(26))
# disp.plot()


In [ ]:
max(edge_label)

In [ ]:
num_classes = cm_node.shape[0]
class_accuracies = {}
for i in range(num_classes):
    class_accuracies[f"Class_{i}"] = cm_node[i, i] / cm_node[i].sum()

# Print accuracy for each class
for key, value in class_accuracies.items():
    print(f"Accuracy for {key}: {value:.8f}")

In [ ]:
num_classes = cm.shape[0]
class_accuracies = {}
for i in range(num_classes-1):
    class_accuracies[f"Class_{i}"] = cm[i, i] / cm[i].sum()

# Print accuracy for each class
for key, value in class_accuracies.items():
    print(f"Accuracy for {key}: {value:.8f}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))  # Change the figure size as needed

# Plot confusion matrix
plt.imshow(cm, cmap='Blues')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center', color='black')
plt.colorbar()

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
acc = (edge_label == edge_pred).sum().item() / edge_label.shape[0]

In [ ]:
acc

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(torch.argmax(pt[3][0], dim=1), pt[3][1])

In [ ]:
pt[0][3]

In [ ]:
import numpy as np
npz = np.load('/home/e19b516g/yejing/data/data_for_graph/S100_R5_Speed_False/test/N9E54_form_5_237_E1183.npz')

In [ ]:
edge_l = npz['edge_labels']

In [ ]:
los = npz['los']

In [ ]:
def find_path(adj_matrix, start, end, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    # Mark the current node as visited and add it to the path
    visited.add(start)
    path.append(start)

    # If the current node is the destination, return the path
    if start == end:
        return path

    # Check neighbors of the current node
    for node, connected in enumerate(adj_matrix[start]):
        if connected and node not in visited:
            # Recursively call find_path for unvisited neighbors
            new_path = find_path(adj_matrix, node, end, visited, path)
            if new_path:
                return new_path

    # If no path is found, backtrack
    path.pop()
    return None

In [ ]:
def set_below_diagonal_to_zero(matrix):
    rows, cols = np.tril_indices(matrix.shape[0], k=-1)  # Get indices below the diagonal
    matrix[rows, cols] = 0  # Set elements below the diagonal to 0
    return matrix

In [ ]:
new_edge_l = set_below_diagonal_to_zero(edge_l)
new_edge_l = np.where(new_edge_l == 1, 0, new_edge_l)

In [ ]:
print(los)
print(edge_l)

In [ ]:
for i in range(los.shape[0]):
    for j in range(i+1, los.shape[0]):
        if los[i][j] == 1 and edge_l[i][j] == 0:
            path = find_path(new_edge_l, i, j)
            print(i,j)
            print(path)
            if path is not None:
                new_edge_l[i][j] = edge_l[path[0]][path[1]]
                print(edge_l[path[0]][path[1]])

In [ ]:
new_edge_l

In [ ]:
los

In [ ]:
l = torch.tensor(los)

In [ ]:
torch.triu(l)

In [ ]:
import numpy as np
new_npz = np.load('/home/e19b516g/yejing/data/data_for_graph/S50_R5_equation/train/N10E60_form_5_769_E3845.npz')

In [ ]:
new_npz['edge_labels']

In [ ]:
new_npz['los']

In [ ]:
from Preprocessing.load import load_gt
inkml = '/home/e19b516g/yejing/data/data_for_graph/INKML/train/CROHME2023_train/form_002_E12.inkml'
lg = '/home/e19b516g/yejing/data/data_for_graph/LG/train/CROHME2023_train/form_002_E12.lg'
s, sl, el, los = load_gt(inkml, lg)

In [ ]:
los_b = torch.zeros(100, 100)
start = 0
los_b[start:start+los.shape[0], start:start+los.shape[0]] = los

In [ ]:
label_b = torch.zeros(100, 100)
label_b[start:start+los.shape[0], start:start+los.shape[0]] = torch.tensor(npz['edge_labels'])

In [ ]:
torch.set_printoptions(threshold=np.inf)
print(label_b)

In [ ]:
print(los_b.shape)

In [ ]:
import os
import numpy as np
for _, _, files in os.walk('/home/e19b516g/yejing/data/data_for_graph/S150_R10/train'):
    for file in files:
        if file.endswith('.npz'):
            npz = np.load(os.path.join('/home/e19b516g/yejing/data/data_for_graph/S150_R10/train', file))
            label = npz['edge_labels']
            print(label.max())

In [ ]:
from Dataset.Dataset import CROHMEDataset
ds = CROHMEDataset(
    data_type='train',
    root_path='/home/e19b516g/yejing/data/data_for_graph/S100_R10',
    batch_size=128,
    max_node=-1
    )

In [ ]:
def edge_filter(edges_emb, edges_label, los):
    los = los.squeeze().fill_diagonal_(0)
    los = torch.triu(los)
    indices = torch.nonzero(los.reshape(-1)).squeeze()
    edges_label = edges_label[indices]
    edges_emb = edges_emb.reshape(-1, edges_emb.shape[-1])[indices]
    return edges_emb, edges_label

In [ ]:
# strokes, relations, los, s_label, r_label = ds.__getitem__(20)
for i in range(ds.__len__()):
    strokes, relations, los, s_label, r_label = ds.__getitem__(i)
    _, new_edge_l = edge_filter(relations, r_label.reshape(-1), los)
    print(torch.where(new_edge_l == 0, 1, 0).sum())
    # calculate number of 0 in new_edge_l

In [ ]:
import os
import numpy as np
import torch
npz_path = '/home/e19b516g/yejing/data/data_for_graph/S100_R10/train/'
for _, _, files in os.walk(npz_path):
    for file in files:
        if file.endswith('.npz'):
            print(file)
            npz = np.load(os.path.join(npz_path, file))
            new_los = torch.triu(torch.tensor(npz['los']))
            indices = torch.nonzero(new_los.reshape(-1)).squeeze()
            label = torch.tensor(npz['edge_labels']).reshape(-1)
            new_label = label[indices]
            print(new_label.max())
            print(torch.where(new_label == 0, 1, 0).sum())

In [ ]:
p = os.path.join(npz_path, 'N8E56_formulaire028-equation062.npz')
npz_p = np.load(p)

In [ ]:
npz_p['edge_labels']

In [ ]:
npz_p['los']

In [ ]:
import torch
x = torch.rand(3,100)
x_re = x.repeat(1, 3).reshape(3, 3, 100)

In [ ]:
x_n = x_re.fill_diagonal_(0)

In [ ]:
t1 = torch.rand(5, 3, 3)
t2 = t1 * (1-torch.eye(3, 3).repeat(5, 1, 1))

In [ ]:
t = torch.zeros(5, 3, 3)
t + torch.eye(3, 3).repeat(5, 1, 1)

In [ ]:
# tensor 3*4, the value of the first column is 1, the value of the second column is 2, and so on
test = torch.arange(1, 5).repeat(3, 1).T
test[0]

In [ ]:
new_los = torch.zeros(128, 128)
los = torch.rand(16, 8, 8)
for i in range(los.shape[0]):
    new_los[i*los.shape[1]:(i+1)*los.shape[1], i*los.shape[1]:(i+1)*los.shape[1]] = los[i]

In [ ]:
new_los[5:14, 5:14]

In [ ]:

indices = torch.arange(16) * 8
new_los[indices[:, None], indices] = los.view(-1, 8, 8)

In [ ]:
t = torch.rand(3, 3, 5)
print(t)
torch.diagonal(t).transpose(0, 1)

In [ ]:
def calculate_piece_positions(shape, padding_size):
    total_length = shape
    start = 0
    positions = []
    
    while start < total_length:
        end = min(start + 8, total_length)
        positions.append([start, end])
        start = end
    
    if padding_size > 0:
        positions[0][0] -= padding_size  # Adjust the start position for the first piece
        
        # Adjust the end position for the last piece if there's padding
        last_piece_index = len(positions) - 1
        positions[last_piece_index][1] += padding_size
        
        # Adjust the end position of the last piece if it exceeds the total length
        if positions[last_piece_index][1] > total_length:
            positions[last_piece_index][1] = total_length
    
    return positions

In [ ]:
def padding(name, shape, pad):
    list = []
    list.append([name, shape, 0, 8 - pad])
    start = 8 - pad
    nb = math.ceil((shape + pad) / 8)
    for i in range(nb - 1):
        if i == nb - 2:
            end = shape
        else:
            end = start + 8
        list.append([name, shape, start, end])
        start += 8
    return list

In [ ]:
import math
padding('miaom',80, 5)

In [ ]:
t = torch.rand(10, 10)
t[8:10,:].shape

In [ ]:
import numpy as np
npz = np.load('/home/e19b516g/yejing/data/data_for_graph/S100_R10/test/N10E42_form_315_E2513.npz')

In [ ]:
torch.from_numpy(npz['edges_emb']).shape

In [ ]:
def load_batch(batch):
    strokes_emb, edges_emb, los, strokes_label, edges_label = batch
    strokes_emb = strokes_emb
    strokes_emb = strokes_emb.reshape(strokes_emb.shape[0]*strokes_emb.shape[1], strokes_emb.shape[2], strokes_emb.shape[3])
    edges_emb = edges_emb.squeeze(0).reshape(edges_emb.shape[0],edges_emb.shape[1], edges_emb.shape[2], edges_emb.shape[3]* edges_emb.shape[4])
    strokes_label = strokes_label.squeeze(0).long().reshape(-1)
    edges_label = edges_label.squeeze(0).long()
    # los = los.squeeze(0).fill_diagonal_(1).unsqueeze(-1)
    los = los + torch.eye(los.shape[1], los.shape[2]).repeat(los.shape[0], 1, 1)
    new_los = torch.zeros((los.shape[0]*los.shape[1], los.shape[0]*los.shape[2]))
    new_edges_label = torch.zeros((edges_label.shape[0]*edges_label.shape[1], edges_label.shape[0]*edges_label.shape[2])).long()
    new_edges_emb = torch.zeros((edges_emb.shape[0]*edges_emb.shape[1], edges_emb.shape[0]*edges_emb.shape[2], edges_emb.shape[3]))
    for i in range(los.shape[0]):
        new_los[i*los.shape[1]:(i+1)*los.shape[1], i*los.shape[1]:(i+1)*los.shape[1]] = los[i]
        new_edges_label[i*edges_label.shape[1]:(i+1)*edges_label.shape[1], i*edges_label.shape[1]:(i+1)*edges_label.shape[1]] = edges_label[i]
        new_edges_emb[i*edges_emb.shape[1]:(i+1)*edges_emb.shape[1], i*edges_emb.shape[1]:(i+1)*edges_emb.shape[1]] = edges_emb[i]
    return strokes_emb, new_edges_emb, new_los.unsqueeze(-1), strokes_label, new_edges_label.reshape(-1)

In [ ]:
from Dataset.Datamodule import CROHMEDatamodule
dm = CROHMEDatamodule(
    root_path = '/home/e19b516g/yejing/data/data_for_graph/S100_R10',
    shuffle = True,
    num_workers = 0,
    batch_size = 8,
    max_node = 8,
    reload_dataloaders_every_n_epochs=1
)
dm.setup('fit')
for batch in dm.train_dataloader():
    try:
        r = load_batch(batch)
    except:
        print('error')
        print(batch[0].shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
strokes_emb = np.load('/home/e19b516g/yejing/data/data_for_graph/S150_R10/train/strokes_emb/N12E84_formulaire017-equation059.npy')

In [ ]:
stroke = strokes_emb[5]
plt.figure(figsize=(6, 6))
plt.plot(stroke[:, 0], -stroke[:, 1], marker='o', linestyle='-')
plt.title('Stroke Plot')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.grid(True)
plt.show()

In [ ]:
import torch
t = torch.rand(3, 3, 5).long()


In [ ]:
tt = torch.where(t == 0, 1, 0)

In [ ]:
data = torch.rand(8, 3, 3, 10)
mean = data.mean(dim=(0,1,2), keepdim=True)
std = data.std(dim=(0,1, 2), keepdim=True)

In [ ]:
print(data.shape)
print(mean.shape)
print(std.shape)

In [ ]:
print(((data- mean)/std).shape)

In [ ]:
los = torch.eye(3, 3)
am = torch.ones(3, 3)
print(am  los)

In [ ]:
import numpy as np

def calculate_curvature(stroke):
    # Check for minimum number of points required for curvature calculation
    if len(stroke) < 3:
        raise ValueError("At least three points are required for curvature calculation")

    # Convert the list of points to numpy array for easy calculations
    stroke = np.array(stroke)

    # Calculate first derivatives
    dx = np.gradient(stroke[:, 0])
    dy = np.gradient(stroke[:, 1])

    # Calculate second derivatives
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)

    # Calculate curvature using the formula: curvature = |dx * ddy - dy * ddx| / (dx^2 + dy^2)^(3/2)
    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2)**(3/2)

    return curvature

# Example stroke (list of points)
stroke = [(1, 5), (3, 2), (5, 6), (7, 8), (9, 10)]

# Calculate curvature for the stroke
curvatures = calculate_curvature(stroke)

print("Curvatures at each point:")
print(curvatures)


In [ ]:
strokes_emb = np.load('/home/e19b516g/yejing/data/data_for_graph/S100geo_feat/val/strokes_emb/N4E10_form_080_E638.npy')
print(strokes_emb.shape)

In [ ]:
import os
src_path = '/home/e19b516g/yejing/data/data_for_graph/S150_R10/'
tgt_path = '/home/e19b516g/yejing/data/data_for_graph/S150_R10_new/'
for root, dirs, files in os.walk(src_path):
    for file in files:
        if file.endswith('.npz'):
            tag = root.split('/')[-1]
            file_name = file.split('.')[0]
            data = np.load(os.path.join(root, file))
            if not os.path.exists(os.path.join(tgt_path, tag, 'strokes_emb')):
                os.makedirs(os.path.join(tgt_path, tag, 'strokes_emb'))
            np.save(os.path.join(tgt_path, tag, 'strokes_emb', file_name), data['strokes_emb'])
            if not os.path.exists(os.path.join(tgt_path, tag, 'edges_emb')):
                os.makedirs(os.path.join(tgt_path, tag, 'edges_emb'))
            np.save(os.path.join(tgt_path, tag, 'edges_emb', file_name), data['edges_emb'])
            if not os.path.exists(os.path.join(tgt_path, tag, 'los')):
                os.makedirs(os.path.join(tgt_path, tag, 'los'))
            np.save(os.path.join(tgt_path, tag, 'los', file_name), data['los'])
            if not os.path.exists(os.path.join(tgt_path, tag, 'stroke_labels')):
                os.makedirs(os.path.join(tgt_path, tag, 'stroke_labels'))
            np.save(os.path.join(tgt_path, tag, 'stroke_labels', file_name), data['stroke_labels'])
            if not os.path.exists(os.path.join(tgt_path, tag, 'edge_labels')):
                os.makedirs(os.path.join(tgt_path, tag, 'edge_labels'))
            np.save(os.path.join(tgt_path, tag, 'edge_labels', file_name), data['edge_labels'])


In [ ]:
from Preprocessing.LG.lg import Lg
lg = Lg('/home/e19b516g/yejing/data/data_for_graph/LG/val/CROHME2023_val/form_5_648_E3236.lg').segmentGraph()

In [ ]:
lg_root_path = '/home/e19b516g/yejing/data/data_for_graph/LG/'
symbol_list = []
for root, dirs, files in os.walk(lg_root_path):
    for file in files:
        if file.endswith('.lg'):
            lg = Lg(os.path.join(root, file)).segmentGraph()
            for key in lg[0]:
                symbol_list.append(lg[0][key][1][0])

In [ ]:
print(symbol_list)

In [ ]:
from collections import Counter
occurrences = Counter(symbol_list).most_common()
lg_dic = []
for xxx in occurrences:
    lg_dic.append(xxx[0])
# Print occurrences of each string
# for string, count in occurrences.items():
#     print(f"'{string}' occurs {count} times")


In [ ]:
set(lg_dic) -set(ink_dic)

In [ ]:
import xml.etree.ElementTree as ET
doc_namespace = "{http://www.w3.org/2003/InkML}"
def load_inkml(file_path):
    strokes = []
    labels = []
    tree = ET.parse(file_path)
    root = tree.getroot()
    last_stroke = []
    dic = {}
    for trace_tag in root.findall(doc_namespace + 'traceGroup'):
        for trace_tag in trace_tag.findall(doc_namespace + 'traceGroup'):
            for annotation in trace_tag.findall(doc_namespace + 'annotation'):
                label = annotation.text
            for traceview in trace_tag.findall(doc_namespace + 'traceView'):
                s_id = traceview.get('traceDataRef')
                dic[s_id] = label
    for trace_tag in root.findall(doc_namespace + 'trace'):
        points = []
        last_point = (0, 0)
        for coord in (trace_tag.text).replace('\n', '').split(','):
            this_point = (float(coord.strip().split(' ')[0]), float(coord.strip().split(' ')[1]))
            if this_point != last_point:
                points.append(this_point)
                last_point = this_point
        for coord in trace_tag.items():
            id = coord[1]
        if last_stroke != points:
            strokes.append(points)
            points_np = np.array(points)
            if np.isnan(points_np).any() == True:
                print(file_path)
            if dic.__contains__(id):
                labels.append(dic[id])
            else:
                labels.append('None')
                dic[id] = 'None'
        last_stroke = points
    return labels

In [ ]:
inkml_root_path = '/home/e19b516g/yejing/data/data_for_graph/INKML/'
symbol_list_ink = []
for root, dirs, files in os.walk(inkml_root_path):
    for file in files:
        if file.endswith('.inkml'):
            labels = load_inkml(os.path.join(root, file))
            symbol_list_ink = symbol_list_ink + labels

In [ ]:
occurrences = Counter(symbol_list_ink).most_common()
ink_dic = []
for xxx in occurrences:
    print(xxx[0])
    ink_dic.append(xxx[0])

In [ ]:
print(ink_dic)

In [ ]:
import torch
tt = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tt +1

In [ ]:
def get_mask(edge_labels, start, end):
    mask = torch.ones(end - start)
    max_len = edge_labels.shape[0]
    for i in range(end , start, -1):
        if i == max_len:
            break
        elif (edge_labels[i-1, i] == 1):
            pos = i - start - 1
            mask[pos] = 0
        else:
            break
    for i in range(start, end-1):
        if (edge_labels[i-1, i] == 1):
            pos = i - start
            mask[pos] = 0
        else:
            break
    return mask



In [ ]:
import torch
edge_labels = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 1, 0, 0, 0, 0],[0, 0, 1, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 1],[0, 0, 0, 0, 0, 0, 1, 0]])
max_node = 5
start = 0
end = 3

In [ ]:
get_mask(edge_labels, 1, 5)

In [ ]:
edge_labels

In [ ]:
for i in range(8, 0, -1):
    print(i)

In [ ]:
result_path = './results/test/'
import torch
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
pred_edge = torch.zeros(0)
pred_node = torch.zeros(0)
gt_edge = torch.zeros(0)
gt_node = torch.zeros(0)
for root, _, files in os.walk(result_path):
    for file in files:
        pt = torch.load(os.path.join(root, file), map_location=torch.device('cpu'))
        stroke_emb = pt[0]
        edge_emb = pt[1]
        stroke_label = pt[2]
        edge_label = pt[3]
        indices = torch.nonzero(edge_label.reshape(-1))
        edge_emb = edge_emb.reshape(-1, edge_emb.shape[-1])[indices]
        edge_label = edge_label.reshape(-1)[indices]
        pred_edge = torch.cat((pred_edge, edge_emb), dim=0)
        pred_node = torch.cat((pred_node, stroke_emb), dim=0)
        gt_edge = torch.cat((gt_edge, edge_label), dim=0)
        gt_node = torch.cat((gt_node, stroke_label), dim=0)
pred_edge = pred_edge.squeeze(1)
cm = confusion_matrix(gt_edge, torch.argmax(pred_edge, dim=1))
cm_node = confusion_matrix(gt_node, torch.argmax(pred_node, dim=1))




In [ ]:
unique_integers = set(torch.argmax(pred_edge, dim = 1).view(-1).numpy())
num_unique_integers = len(unique_integers)
print(unique_integers)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gt_node, torch.argmax(pred_node, dim = 1)))
print(accuracy_score(gt_edge, torch.argmax(pred_edge, dim = 1)))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))  # Change the figure size as needed

# Plot confusion matrix
plt.imshow(cm, cmap='Blues')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center', color='black')
plt.colorbar()

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
num_classes = cm.shape[0]
class_accuracies = {}
for i in range(num_classes):
    class_accuracies[f"Class_{i}"] = cm[i, i] / cm[i].sum()

# Print accuracy for each class
for key, value in class_accuracies.items():
    print(f"Accuracy for {key}: {value:.8f}")

In [ ]:
pt = torch.load('./results/test/N11E72_UN19_1028_em_394.pt', map_location=torch.device('cpu'))
stroke_emb = pt[0]
edge_emb = pt[1]
stroke_label = pt[2]
edge_label = pt[3]

print(torch.argmax(stroke_emb, dim=1))
print(stroke_label)

In [ ]:
print(torch.argmax(edge_emb, dim=2))
print(edge_label.int())

In [ ]:
edge_emb[1,4] + edge_emb[0,4]

In [ ]:
def connected_components_mask(adjacency_matrix):
        adjacency_matrix = adjacency_matrix.cpu()
        # batch_nb = adjacency_matrix.shape[0]
        num_nodes = adjacency_matrix.shape[0]
        visited = torch.zeros(num_nodes, dtype=bool)
        def dfs(node, component):
            visited[node] = True
            component.append(node)
            for neighbor in range(num_nodes):
                if adjacency_matrix[node][neighbor] == 1 and not visited[neighbor]:
                    dfs(neighbor, component)
        components = []
        for node in range(num_nodes):
            if not visited[node]:
                component = []
                dfs(node, component)
                components.append(component)
        max_node = max(max(component) for component in components) + 1
        mask = torch.zeros(len(components), max_node, dtype=torch.int)
        for i, component in enumerate(components):
            for node in component:
                mask[i][node] = 1
        return mask, components

def sub_graph_pooling(node_feat, edge_feat):
    am = torch.argmax(edge_feat, dim=2)
    print(am)
    # am = torch.zeros(node_feat.shape[0], node_feat.shape[0])
    mask = connected_components_mask(am)
    print(mask)



    node_feat_repeat = node_feat.unsqueeze(0).repeat(mask.shape[0], 1,1)
    node_out = torch.sum(node_feat_repeat * mask.unsqueeze(-1), dim=1)/ mask.sum(dim=1).unsqueeze(-1)
    print(node_out.shape)

    edge_out = torch.zeros(mask.shape[0], mask.shape[0], edge_feat.shape[-1])

    # node_out = torch.matmul(avg_pooled_lines.t().float(), mask.float()).t()

    edge_feat_repeat = edge_feat.unsqueeze(0).repeat(mask.shape[0], 1, 1, 1)
    avg_pooled_lines = torch.sum(edge_feat_repeat * mask.unsqueeze(1).unsqueeze(-1), dim=2)/ mask.sum(dim=1).unsqueeze(1).unsqueeze(-1)
    print(edge_feat_repeat.shape)
    print(mask.unsqueeze(1).unsqueeze(-1).shape)
    print(avg_pooled_lines.shape)
    # edge_out = torch.matmul(avg_pooled_lines.permute(2,1,0).unsqueeze(1), mask.float().t().unsqueeze(2))
    # print(torch.argmax(avg_pooled_lines,dim=2))
    # print(avg_pooled_lines.shape)
    edge_out_repeat = avg_pooled_lines.squeeze(-1).permute(2,1,0).unsqueeze(0).repeat(mask.shape[0],1, 1, 1)
    edge_out_repeat = edge_out_repeat.permute(0,3,2,1)
    print(edge_out_repeat.shape)
    print(mask.unsqueeze(1).unsqueeze(-1).shape)
    edge_out = torch.sum(edge_out_repeat * mask.unsqueeze(1).unsqueeze(-1), dim=2)/ mask.sum(dim=1).unsqueeze(1).unsqueeze(-1)
    print(edge_out.shape)


    return node_out, edge_out

In [ ]:
node_out, edge_out = sub_graph_pooling(stroke_emb, edge_emb)

In [ ]:
torch.argmax(node_out, dim=1)

In [ ]:
def lg2symlg(edge_label, componets):
    # node_out, edge_out = sub_graph_pooling(edge_emb)
    # am = edge_label.where(edge_label == 1, 1, 0)
    # mask = connected_components_mask(am)
    symlg = torch.zeros(len(componets), len(componets))
    for i in range(len(componets)):
        x = edge_label[:,componets[j]]
        y = edge_label[componets[j],:]
        combine(x)
        for j in componets[i]:
            if edge_label[i][j] != 0 and edge_label[i][j] != 1:
                symlg[i][j] = edge_label[i][j]
                break
    return symlg

In [ ]:
edge_label[:,5:].shape
torch.max(edge_label[:,5:],dim=1)

In [ ]:
def combine(edge_label,componets):
    for i in range(len(componets)):
        componet = componets[i]
        x = edge_label[:,componet]
        x = torch.where(x==1, 0,x)
        before_x = edge_label[:,:componet[0]]
        after_x = edge_label[:,componet[-1] +1:]
        x_new = torch.max(x,dim=1).values.unsqueeze(-1)
        edge_label = torch.cat((before_x, x_new, after_x), dim=1)
        y = edge_label[componet,:]
        y = torch.where(y==1, 0,y)
        y_new = torch.max(y, dim=0).values.unsqueeze(0)
        before_y = edge_label[:componet[0],:]
        after_y = edge_label[componet[-1]+1 :,:]

        edge_label = torch.cat((before_y, y_new, after_y), dim=0)
        gap = len(componet) - 1
        for j in range(len(componets)):
            for k in range(len(componets[j])):
                componets[j][k] = componets[j][k] - gap
        print(componets)
        print(edge_label.shape)
        
    return edge_label

In [ ]:
_, com = connected_components_mask(torch.argmax(edge_emb, dim=2))
new = combine(edge_label, com)

In [ ]:
print(new)

In [ ]:
print(new)
print(torch.argmax(edge_out, dim=2))


In [ ]:
def find_duplicates_except_zero(list):
    seen = {}
    duplicate_indices = []

    for i in range(len(list)):
        if list[i] != 0:
            if list[i]  not in seen:
                seen[list[i]] = i
            else:
                # duplicate_indices[seen[list[i]]] = i
                duplicate_indices.append([seen[list[i]], i])
    grouped_dict = {}
    for sublist in duplicate_indices:
        key = sublist[0]
        value = sublist[1:]
        grouped_dict.setdefault(key, []).extend(value)
    out = []
    for key in grouped_dict:
        out.append([key] + grouped_dict[key])
    return out

In [ ]:
def has_duplicates_except_zero(tensor):
    unique_values, _ = torch.unique(tensor, return_counts=True)
    print(unique_values)
    # Check if there are any duplicates (excluding 0)
    return (unique_values != 0).any() and (len(unique_values) != len(tensor))

In [ ]:
tensor_with_duplicates = [1, 1, 3, 3, 4, 4,1]
find_duplicates_except_zero(tensor_with_duplicates)

In [ ]:
def remove_repeat(edge_pred):
    for i in range(edge_pred.shape[0]):
        repeat_x = find_duplicates_except_zero(edge_pred[i,:].tolist())
        repeat_y = find_duplicates_except_zero(edge_pred[:,i].tolist())
        if repeat_x != []:
            for j in repeat_x:
                near = [abs(x - i) for x in j]
                keep = min(near)
                for x in j:
                    if x != keep:
                        edge_pred[i,x] = 0
        if repeat_y != []:
            for j in repeat_y:
                near = [abs(x - i) for x in j]
                keep = min(near)
                for x in j:
                    if x != keep:
                        edge_pred[x,i] = 0
    return edge_pred

In [ ]:
remove_repeat(torch.argmax(edge_out, dim=2))

In [ ]:
is_all_zeros = torch.all(edge_out == 0).item()

In [ ]:
def is_directed_tree(adjacency_matrix):
    # 获取邻接矩阵的维度
    num_nodes = adjacency_matrix.size(0)
    
    # 条件1: 节点数量等于边的数量加1
    num_edges = torch.sum(adjacency_matrix)
    condition1 = (num_nodes == num_edges + 1)

    # 使用深度优先搜索（DFS）判断有向图是否连通
    def dfs(node, visited):
        visited[node] = True
        for neighbor in range(num_nodes):
            if adjacency_matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor, visited)

    # 初始化访问标记数组
    visited = torch.zeros(num_nodes, dtype=torch.bool)

    # 从第一个节点开始进行DFS
    dfs(0, visited)

    # 条件2: 有向图是连通的
    condition2 = visited.all()

    # 条件3: 有向图中没有环
    condition3 = (torch.sum(adjacency_matrix) == num_nodes - 1)

    # 如果同时满足三个条件，则结构是有向树（DAG）
    return condition1 and condition2 and condition3

In [ ]:
tree = torch.tensor(
        [[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
)
mask_tree = torch.where(tree == 0, 0, 1)

In [ ]:
is_directed_tree(mask_tree)

In [ ]:
X = np.zeros((0,40))

In [ ]:
y = np.zeros((0))
print(y)

In [1]:
import torch
import os
import numpy as np
root_path = '/home/e19b516g/yejing/data/data_for_graph/S150_R10'

In [ ]:
def edge_filter(edges_emb, edges_label, los):
        edges_emb = edges_emb.reshape(-1,40)
        edges_label = edges_label.reshape(-1)
        edges_label = torch.where(edges_label < 14, edges_label, 0)
        indices = torch.nonzero(los.reshape(-1)).squeeze().reshape(-1)
        edges_label = edges_label[indices]
        edges_emb = edges_emb[indices]
        return edges_emb, edges_label

def make_data(path, tag):
        X = torch.zeros((0,40))
        y = torch.zeros(0)
        for root, _, files in os.walk(os.path.join(path,tag,'los')):
                for file in files:
                        los = torch.from_numpy(np.load(os.path.join(root,file)))
                        edge_emb = torch.from_numpy(np.load(os.path.join(path,tag,'edges_emb',file)))
                        edge_labels = torch.from_numpy(np.load(os.path.join(path,tag,'edge_labels',file)))
                        edge_emb, edge_labels = edge_filter(edge_emb, edge_labels, los)
                        X = torch.cat((X, edge_emb))
                        y = torch.cat((y, edge_labels))
        torch.save(X, os.path.join(path,tag + '_edge_X.pt'))
        torch.save(y, os.path.join(path,tag + '_edge_y.pt'))


In [ ]:
make_data(root_path,'test')

In [30]:
from Model.MainModel import Edge_emb
class Edge_emb_softmax(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.edge_emb = Edge_emb([40,384,14],0.2)
        self.softmax= torch.nn.Softmax()
    def forward(self,x):
        print(x.shape)
        x = self.edge_emb(x).reshape(-1,14)
        print(x.shape)
        return self.softmax(x)

In [32]:
# from tsai.all import *
# from p_train import *
# import torch
test_y = torch.load(os.path.join(root_path,'alledge', 'test_y.pt')).long()
test_X = torch.load(os.path.join(root_path,'alledge', 'test_X.pt')).float()
# datasets_test = TSDatasets(test_X.float(), test_y)
# dataloader = TSDataLoaders.from_dsets(datasets_test, bs = 250, num_workers=0)
pt_path = './models/edge.pth'
model = Edge_emb_softmax()
model.load_state_dict(torch.load(pt_path))
# model.eval()
with torch.no_grad():
    pred= model(test_X)
# learn = ts_learner(dataloader, model, loss_func=nn.CrossEntropyLoss(), metrics=accuracy, weights_path=pt_path)
# preds, _ = learn.get_preds()

torch.Size([448496, 40])


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [26]:
model

Edge_emb_softmax(
  (edge_emb): Edge_emb(
    (linear0): Linear(in_features=40, out_features=384, bias=True)
    (bn0): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation0): LeakyReLU(negative_slope=0.01)
    (dropout0): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=384, out_features=14, bias=True)
    (bn1): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation1): LeakyReLU(negative_slope=0.01)
    (dropout1): Dropout(p=0.2, inplace=False)
  )
  (softmax): Softmax(dim=None)
)

In [13]:
test_X.shape

torch.Size([448496, 40])

<All keys matched successfully>